In [ ]:
# Artificial Data
import numpy as np
import pandas as pd
import unsupervised_scripts

## Initialize start and end of hyperspectral sweep; num_samp is calculated from image stack
wavenum_1 = int(input('Enter first wavenumber/higher wavelength (Default=2700)): ').strip() or '2700')
wavenum_2 = int(input('Enter last wavenumber/lower wavelength (Default=3100): ').strip() or '3100')
num_samp = int(input('Enter HSI step size: ').strip()) 
ch_start = int(np.floor(num_samp/((wavenum_2-wavenum_1)/(2800-wavenum_1))))
og_wavenum_1 = 2700
og_wavenum_2 = 3100
og_num_samp = 81
og_ch_start = int(np.floor(og_num_samp/((og_wavenum_2-og_wavenum_1)/(2800-og_wavenum_1))))
background_df = pd.read_csv('water_HSI_76.csv')
molecule_df = pd.read_excel('lipid_subtype.xlsx')
shift = 15

## Training Directory (different for everyone)
path_dir = 'D:/Shi_Lab/Spectral_Clustering/'
data_dir = 'D:/Shi_Lab/Spectral_Clustering/ML/'
sample_dir = data_dir + 'training_data/'+str(num_samp)+'/'

artificial_dataset = unsupervised_scripts.artificial_dataset(wavenum_1, wavenum_2, num_samp, ch_start, background_df, shift)
[_, noise_scale_vec, bg_scale_vec, ratio_scale_vec] = artificial_dataset.save_srs_params(sample_dir)
[mol_norm, mol_names] = artificial_dataset.molecule_dataset(molecule_df)
artificial_dataset.save_artificial_dataset(mol_norm, noise_scale_vec, bg_scale_vec, ratio_scale_vec, noise_param=1, num_values=1000, num_repeats=20, name='artificial_training_data-')

100%|██████████| 26/26 [00:00<?, ?it/s]


In [2]:
# DAE Data
import numpy as np
import pandas as pd
import unsupervised_scripts

## Initialize start and end of hyperspectral sweep; num_samp is calculated from image stack
wavenum_1 = int(input('Enter first wavenumber/higher wavelength (Default=2700)): ').strip() or '2700')
wavenum_2 = int(input('Enter last wavenumber/lower wavelength (Default=3100): ').strip() or '3100')
num_samp = int(input('Enter HSI step size: ').strip()) 
ch_start = int(np.floor(num_samp/((wavenum_2-wavenum_1)/(2800-wavenum_1))))
og_wavenum_1 = 2700
og_wavenum_2 = 3100
og_num_samp = 81
og_ch_start = int(np.floor(og_num_samp/((og_wavenum_2-og_wavenum_1)/(2800-og_wavenum_1))))
background_df = pd.read_csv('water_HSI_76.csv')
molecule_df = pd.read_excel('lipid_subtype.xlsx')
shift = 15

## Training Directory (different for everyone)
path_dir = 'D:/Shi_Lab/Spectral_Clustering/'
data_dir = 'D:/Shi_Lab/Spectral_Clustering/ML/'
sample_dir = data_dir + 'training_data/'+str(num_samp)+'/'

artificial_dataset = unsupervised_scripts.artificial_dataset(wavenum_1, wavenum_2, num_samp, ch_start, background_df, shift)
[_, noise_scale_vec, bg_scale_vec, ratio_scale_vec] = artificial_dataset.save_srs_params(sample_dir)
[mol_norm, mol_names] = artificial_dataset.molecule_dataset(molecule_df)
artificial_dataset.save_artificial_dataset(mol_norm, noise_scale_vec, bg_scale_vec, ratio_scale_vec, noise_param=1, num_values=10000, num_repeats=20, name='denoising_training_data-')


100%|██████████| 26/26 [00:00<00:00, 381.63it/s]
